<a href="https://colab.research.google.com/github/itsGab/previsao_uti_em_covid/blob/main/notebooks/4_MACHINE_LEARNING_analise_de_modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align='center'> Previsão de Admissão na UTI por COVID-19 </h1>
<h2 align='center'> Notebook De MACHINE LEARNING</h2>

In [102]:
!pip install -U lazypredict

# IMPORTAÇÕES (data & libs)

In [104]:
# imports
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score, classification_report, recall_score
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.linear_model import LogisticRegression, RidgeClassifier, RidgeClassifierCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.model_selection import KFold, StratifiedShuffleSplit, RepeatedKFold, StratifiedKFold, RepeatedStratifiedKFold
import lightgbm as lgb
from lazypredict.Supervised import LazyClassifier
from sklearn.dummy import DummyClassifier

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
sns.set_theme(style="whitegrid")

import warnings
warnings.filterwarnings('ignore')

# importação dos dados
data_cleaned = 'https://github.com/itsGab/previsao_uti_em_covid/raw/main/dados/Dados_limpos'

# ABRINDO O CONJUNTO DE DADOS


In [105]:
# lendo os dados com pandas
df = pd.read_csv(data_cleaned, index_col=0)

# mostrando o formato e tamanho do dataframe
print(f'formato: {df.shape[0]} linhas, {df.shape[1]} colunas')
print(f'tamanho: {df.size}')

# mostrando o cabeçalho do dataframe
df.head()

formato: 296 linhas, 60 colunas
tamanho: 17760


,ICU,AGE_ABOVE65,AGE_PERCENTIL,GENDER,DISEASE_GROUPING_1,DISEASE_GROUPING_2,DISEASE_GROUPING_3,DISEASE_GROUPING_4,DISEASE_GROUPING_5,DISEASE_GROUPING_6,HTN,IMMUNOCOMPROMISED,OTHER,ALBUMIN_MAX,BE_VENOUS_MAX,BIC_ARTERIAL_MAX,BIC_VENOUS_MAX,BILLIRUBIN_MAX,BLAST_MAX,CALCIUM_MAX,CREATININ_MAX,FFA_MAX,GGT_MAX,GLUCOSE_MAX,HEMOGLOBIN_MAX,INR_MAX,LACTATE_MAX,LINFOCITOS_MAX,NEUTROPHILES_MAX,P02_VENOUS_MAX,PC02_ARTERIAL_MAX,PC02_VENOUS_MAX,PCR_MAX,PH_ARTERIAL_MAX,PH_VENOUS_MAX,PLATELETS_MAX,POTASSIUM_MAX,SAT02_ARTERIAL_MAX,SAT02_VENOUS_MAX,SODIUM_MAX,TGO_MAX,TGP_MAX,TTPA_MAX,UREA_MAX,DIMER_MAX,BLOODPRESSURE_DIASTOLIC_MIN,RESPIRATORY_RATE_MIN,TEMPERATURE_MIN,OXYGEN_SATURATION_MIN,BLOODPRESSURE_DIASTOLIC_MAX,BLOODPRESSURE_SISTOLIC_MAX,HEART_RATE_MAX,RESPIRATORY_RATE_MAX,TEMPERATURE_MAX,OXYGEN_SATURATION_MAX,BLOODPRESSURE_DIASTOLIC_DIFF_REL,BLOODPRESSURE_SISTOLIC_DIFF_REL,HEART_RATE_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL
PATIENT_VISIT_IDENTIFIER,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1,1,5,0,0,0,0,0,1,1,0,0,1,0.61,-1.00,-0.32,-0.32,-0.94,-1.00,0.18,-0.87,-0.74,-0.95,-0.89,0.11,-0.93,1.00,-0.91,-0.87,-0.70,-0.78,-0.75,-0.88,0.23,0.36,-0.54,-0.52,0.94,0.35,-0.03,-1.00,-0.99,-0.83,-0.84,-0.99,0.24,-0.50,0.21,0.90,-0.25,-0.46,-0.43,-0.64,-0.42,0.74,-1.00,-1.00,-1.00,-1.00,-1.00
2,1,0,0,0,0,0,0,0,0,0,0,0,1,0.61,-1.00,-0.32,-0.32,-0.94,-1.00,0.36,-0.91,-0.74,-0.96,-0.78,0.16,-0.96,1.00,-0.91,-0.41,-0.70,-0.78,-0.75,-0.94,0.23,0.36,-0.40,-0.70,0.94,0.35,0.09,-1.00,-0.99,-0.85,-0.84,-0.98,-0.53,-0.71,0.60,0.96,-0.44,-0.49,0.00,-0.58,0.10,1.00,-0.52,-0.35,-0.75,-0.76,-1.00
3,0,0,3,1,0,0,0,0,0,0,0,1,1,-0.26,-1.00,-0.32,-0.32,-0.97,-1.00,0.33,-0.97,-0.19,-0.32,-0.89,-0.22,-0.96,-0.83,-0.84,-0.78,-0.63,-0.78,-0.78,-0.50,0.23,0.36,-0.56,-0.78,0.94,0.58,0.20,-0.99,-0.96,-0.85,-0.94,-0.98,0.18,-0.36,0.21,0.88,-0.30,-0.56,-0.63,-0.52,-0.42,0.68,-1.00,-1.00,-1.00,-1.00,-1.00
4,0,0,0,0,0,0,0,0,0,0,0,0,1,0.61,-1.00,-0.32,-0.32,-0.94,-1.00,0.36,-0.91,-0.83,-0.94,-0.85,0.30,-0.96,1.00,-0.64,-0.81,-0.70,-0.78,-0.75,-0.99,0.23,0.36,-0.46,-0.59,0.94,0.35,0.14,-1.00,-0.99,-0.85,-0.90,-1.00,0.44,-0.57,0.54,0.94,-0.08,-0.35,-0.04,-0.58,0.07,0.89,-1.00,-0.88,-0.96,-0.87,-0.95
6,0,1,6,1,0,0,0,0,0,0,0,1,1,0.61,-1.00,-0.32,-0.32,-0.94,-1.00,0.36,-0.94,-0.74,-0.96,-0.89,-0.48,-0.96,1.00,-0.86,-0.95,-0.70,-0.78,-0.75,-0.75,0.23,0.36,-0.64,-0.96,0.94,0.35,-0.26,-1.00,-0.99,-0.85,-0.95,-0.98,-0.01,-0.50,0.49,0.96,-0.45,-0.61,-0.54,-0.64,-0.04,0.89,-1.00,-1.00,-1.00,-1.00,-1.00


# MODELOS

In [106]:
# padronizando o numero do random seed
def semente(num=185542):
    np.random.seed(num)

# Separando dados TREINO e TEST

In [107]:
semente()

X = df.drop(columns=['ICU'])
y = df['ICU']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

# lazyclassifier


In [108]:
semente()

cls = LazyClassifier(predictions=True, random_state=185542, custom_metric=recall_score)
mod, pred = cls.fit(X_train, X_test, y_train, y_test)

mod.sort_values('recall_score', ascending=False)

100%|██████████| 29/29 [00:01<00:00, 16.54it/s]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,recall_score,Time Taken
Model,,,,,,
NearestCentroid,0.73,0.71,0.71,0.73,0.63,0.03
SGDClassifier,0.61,0.60,0.60,0.61,0.59,0.03
PassiveAggressiveClassifier,0.61,0.60,0.60,0.61,0.59,0.03
BernoulliNB,0.73,0.70,0.70,0.73,0.59,0.02
BaggingClassifier,0.68,0.64,0.64,0.67,0.52,0.06
RandomForestClassifier,0.68,0.64,0.64,0.67,0.52,0.26
LGBMClassifier,0.66,0.63,0.63,0.66,0.52,0.07
AdaBoostClassifier,0.65,0.62,0.62,0.65,0.52,0.13
Perceptron,0.55,0.55,0.55,0.56,0.52,0.02


## Escolhendo os modelos


O modelo com melhor desempenho no LazyClassifier, o '**LGMLGBMClassifier**', e mais alguns modelos como: **LogisticRegrission, RandomForestClassifier, RidgeClassifierCV**.

A metrica que estou usando de base é o **recall**, que representa a quantidade de casos positivos foram acertados pelo modelo.

### Modelos para teste

In [109]:
# dicionario de nomes e modelos
modelos = {
    'Random Forest': RandomForestClassifier, 
    'Gradient Boosting': GradientBoostingClassifier,
    'RidgeClassifier': RidgeClassifier,
    'RidgeClassifierCV':  RidgeClassifierCV,
    'AdaBoostClassifier': AdaBoostClassifier,
    'LGBMClassifier': lgb.LGBMClassifier,
    'NearestCentroid': NearestCentroid,
    'LogisticRegression': LogisticRegression,
    'DummyClassifier': DummyClassifier
    }

### Sem estratificar dados de treino e teste

In [110]:
# treino e teste estratificados
semente()
X = df.drop(columns=['ICU'])
y = df['ICU']
X_train, X_test, y_train, y_test = train_test_split(X, y)

# lista para guardando info dos modelos
info_modelos = []

# loop para rodar modelos
for nome in modelos:

    # reproducibilidade
    semente()

    # modelo, fit e predict
    modelo = modelos[nome]()
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)

    # classification report
    dict_report = classification_report(y_test, y_pred, output_dict=True)
    dict_modelo = {'modelo':  nome}
    dict_modelo['recall'] = dict_report['1']['recall']
    dict_modelo['accuracy'] = dict_report['accuracy']
    info_modelos.append(dict_modelo)
    # print(nome,'\n', classification_report(y_test, y_pred))

# data frame dos resultados
info_sem_strat = pd.DataFrame(data=info_modelos).sort_values(by='recall', ascending=False)
info_sem_strat

,modelo,recall,accuracy
5,LGBMClassifier,0.71,0.84
4,AdaBoostClassifier,0.68,0.72
2,RidgeClassifier,0.64,0.81
7,LogisticRegression,0.64,0.81
1,Gradient Boosting,0.61,0.77
3,RidgeClassifierCV,0.61,0.80
0,Random Forest,0.57,0.77
6,NearestCentroid,0.57,0.58
8,DummyClassifier,0.36,0.54


### Sem estratificar dados de treino e teste

In [111]:
# treino e teste estratificados
semente()
X = df.drop(columns=['ICU'])
y = df['ICU']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

info_modelos = []

# loop para rodar modelos
for nome in modelos:

    # reproducibilidade
    semente()

    # modelo, fit e predict
    modelo = modelos[nome]()
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)

    # classification report
    dict_report = classification_report(y_test, y_pred, output_dict=True)
    dict_modelo = {'modelo':  nome}
    dict_modelo['recall'] = dict_report['1']['recall']
    dict_modelo['accuracy'] = dict_report['accuracy']
    info_modelos.append(dict_modelo)
    # print(nome,'\n', classification_report(y_test, y_pred))

# data frame dos resultados
info_com_strat = pd.DataFrame(data=info_modelos).sort_values(by='recall', ascending=False)
info_com_strat

,modelo,recall,accuracy
6,NearestCentroid,0.67,0.65
5,LGBMClassifier,0.56,0.69
0,Random Forest,0.52,0.69
4,AdaBoostClassifier,0.52,0.65
7,LogisticRegression,0.48,0.62
2,RidgeClassifier,0.44,0.58
3,RidgeClassifierCV,0.44,0.62
1,Gradient Boosting,0.41,0.61
8,DummyClassifier,0.30,0.49


Apesar do LGBMClassifier se sair muito bem no primeiro modelo onde não há a estratificação dos dados de treino e teste, ele se saiu mau no segundo, aonde é usado a estratificação na hora de separar os dados de treino e teste, com isso, vamos fazer uma validação cruzado dos dados, para ver como os modelos se saem usando a divisao dos dados (KFold)

In [112]:
def testa_modelo_cv(cv=KFold()):
    '''funcao para rodar cross validation do modelo'''
    # lista para guardando info dos modelos
    info_modelos = []
    # loop para rodar modelos
    for nome in modelos:
        # reproducibilidade
        semente()
        # modelo
        modelo = modelos[nome]()
        # cross validate
        info_cv = cross_validate(modelo, X, y, scoring=['precision', 'recall', 'accuracy', 'f1'],cv=cv)
        cv_recall = info_cv['test_recall'].mean()
        cv_accuracy = info_cv['test_accuracy'].mean()
        cv_precision = info_cv['test_precision'].mean()
        cv_f1 =  info_cv['test_f1'].mean()
        dict_modelo = {'modelo': nome, 'f1':cv_f1, 'precision': cv_precision, 'recall': cv_recall, 'accuracy': cv_accuracy}
        info_modelos.append(dict_modelo)
    return pd.DataFrame(info_modelos).sort_values('recall', ascending=False)

In [113]:
testa_modelo_cv()

,modelo,f1,precision,recall,accuracy
6,NearestCentroid,0.55,0.51,0.62,0.64
4,AdaBoostClassifier,0.56,0.57,0.57,0.68
1,Gradient Boosting,0.56,0.62,0.53,0.70
5,LGBMClassifier,0.53,0.61,0.49,0.69
7,LogisticRegression,0.52,0.62,0.46,0.69
0,Random Forest,0.52,0.67,0.45,0.71
2,RidgeClassifier,0.50,0.59,0.45,0.69
3,RidgeClassifierCV,0.49,0.62,0.41,0.68
8,DummyClassifier,0.40,0.42,0.39,0.58


Aqui vemos o resultado, apesar do modelo NearestCentroid não apresentar a maior accuracy ele apresenta o mais taxa de recall, o que significa que ela está acertando 62% dos casos positivos.

Acredito que essa seria a métrica mais importante, para o modelo, acertar os casos que estão precisando da UTI.

Os resultados do meu modelo foram insatisfatorios, pois um acerto de 62% é um porcentagem muito baixa, pra algo tão sério como uma vida.